In [1]:
import numpy as np

A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9],
              [10, 11, 12],
              [13, 14, 15]])
print("原始矩阵 A:")
print(A)

原始矩阵 A:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]]


In [2]:
# 进行 SVD 分解
U, sigma, Vt = np.linalg.svd(A, full_matrices=False)
print("\n奇异值 sigma:")
print(sigma)


奇异值 sigma:
[3.51826483e+01 1.47690770e+00 3.74442300e-16]


In [4]:
# 保留前 k=1 个奇异值进行降维
k = 1
U_k = U[:, :k]  # 取 U 的前 k 列，因为要保持行数不变
sigma_k = sigma[:k]  # 取前 k 个奇异值
Vt_k = Vt[:k, :]  # 取 Vt 的前 k 行，因为要保持列数不变

# 近似重构矩阵 A,常用于信号or图像筛除噪声
A_approx = U_k @ np.diag(sigma_k) @ Vt_k
print("\n保留前", k, "个奇异值后的近似矩阵 A_approx:")
print(A_approx)

# 计算近似误差
error = np.linalg.norm(A - A_approx, 'fro') / np.linalg.norm(A, 'fro')
print("\n近似误差 (Frobenius 范数相对误差):", error)


保留前 1 个奇异值后的近似矩阵 A_approx:
[[ 1.85152908  2.05208851  2.25264793]
 [ 4.5411984   5.03310541  5.52501242]
 [ 7.23086771  8.01412231  8.7973769 ]
 [ 9.92053702 10.99513921 12.06974139]
 [12.61020633 13.9761561  15.34210588]]

近似误差 (Frobenius 范数相对误差): 0.04194136031471535


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 设置随机种子以便结果可重复
np.random.seed(42)

# 模拟数据：1000 个样本，50 个特征
n_samples = 1000
n_features = 50
X = np.random.randn(n_samples, n_features) * 10  # 随机生成特征数据
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # 模拟二分类标签

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")

# 对训练集进行 SVD 分解
U_train, sigma_train, Vt_train = np.linalg.svd(X_train, full_matrices=False)
print(f"Vt_train 矩阵形状: {Vt_train.shape}")

# 选择保留的奇异值数量 k
k = 10
Vt_k = Vt_train[:k, :]  # 保留前 k 行，形状为 (k, 50)
print(f"保留 k={k} 后的 Vt_k 矩阵形状: {Vt_k.shape}")

# 降维训练集：X_train_reduced = X_train @ Vt_k.T
X_train_reduced = X_train @ Vt_k.T
print(f"降维后训练集形状: {X_train_reduced.shape}")

# 使用相同的 Vt_k 对测试集进行降维：X_test_reduced = X_test @ Vt_k.T
X_test_reduced = X_test @ Vt_k.T
print(f"降维后测试集形状: {X_test_reduced.shape}")

# 训练模型（以逻辑回归为例）
model = LogisticRegression(random_state=42)
model.fit(X_train_reduced, y_train)

# 预测并评估
y_pred = model.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)
print(f"测试集准确率: {accuracy}")

# 计算训练集的近似误差（可选，仅用于评估降维效果）
X_train_approx = U_train[:, :k] @ np.diag(sigma_train[:k]) @ Vt_k
error = np.linalg.norm(X_train - X_train_approx, 'fro') / np.linalg.norm(X_train, 'fro')
print(f"训练集近似误差 (Frobenius 范数相对误差): {error}")

c:\Anaconda\envs\vs\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


训练集形状: (800, 50)
测试集形状: (200, 50)
Vt_train 矩阵形状: (50, 50)
保留 k=10 后的 Vt_k 矩阵形状: (10, 50)
降维后训练集形状: (800, 10)
降维后测试集形状: (200, 10)
测试集准确率: 0.595
训练集近似误差 (Frobenius 范数相对误差): 0.852479929511559


### 6. 实际操作中的注意事项
1. **标准化数据**：在进行 SVD 之前，通常需要对数据进行标准化（均值为 0，方差为 1），以避免某些特征的量纲差异对降维结果的影响。可以使用 `sklearn.preprocessing.StandardScaler`。
   ```python
   from sklearn.preprocessing import StandardScaler
   scaler = StandardScaler()
   X_train_scaled = scaler.fit_transform(X_train)
   X_test_scaled = scaler.transform(X_test)
   ```
   注意：`scaler` 必须在训练集上 `fit`，然后对测试集只用 `transform`，以避免数据泄漏。

2. **选择合适的 $k$**：可以通过累计方差贡献率（explained variance ratio）选择 $k$，通常选择解释 90%-95% 方差的 $k$ 值。代码中可以计算：
   ```python
   explained_variance_ratio = np.cumsum(sigma_train**2) / np.sum(sigma_train**2)
   print(f"前 {k} 个奇异值的累计方差贡献率: {explained_variance_ratio[k-1]}")
   ```

3. **使用 sklearn 的 TruncatedSVD**：`sklearn` 提供了 `TruncatedSVD` 类，专门用于高效降维，尤其适合大规模数据。它直接计算前 $k$ 个奇异值和向量，避免完整 SVD 的计算开销。
   ```python
   from sklearn.decomposition import TruncatedSVD
   svd = TruncatedSVD(n_components=k, random_state=42)
   X_train_reduced = svd.fit_transform(X_train)
   X_test_reduced = svd.transform(X_test)
   print(f"累计方差贡献率: {sum(svd.explained_variance_ratio_)}")